In [1]:
from IPython.display import IFrame, display
filepath = "../Slides/2022_DistributedComputing_Week4.pdf"
IFrame(src=filepath, width=300, height=200,)

In [2]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/11 08:20:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))

In [4]:
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Return any five distinct  zip code in zip_biz_pair

In [5]:
zip_biz_pair.keys().distinct().take(5)

['94124', '94108', '94102', '94133', '94111']

## 2. Return any five distinct city names.

In [6]:
zip_biz_pair.values().map(lambda x : x[-2]).distinct().take(5)

['San Francisco', 'Hayward', 'Redwood+city', 'Daly+city', 'Mexico']

## 3. Return five distinct (zip code, city name) pairs ordered by zip code, where zip code has 5 digits.

In [7]:
distinct_zip_city_pairs = zip_biz_pair.filter(lambda x : len(x[0]) == 5)\
                                      .mapValues(lambda x : x[-2])\
                                      .distinct()\
                                      .sortByKey()

In [8]:
distinct_zip_city_pairs.take(5)

[('10001', 'New+york'),
 ('10002', 'New+york'),
 ('10003', 'New+york'),
 ('10004', 'New+york'),
 ('10004', 'New York')]

## 4. Find the zip associated with the most city names
* which one is more efficient ??
    - zip_associated_with_multiple_cities_1  $vs$ zip_associated_with_multiple_cities_2

#### v1

In [9]:
zip_associated_with_multiple_cities_1 = distinct_zip_city_pairs.map(lambda x : (x[0], 1))\
                                           .groupByKey()\
                                           .mapValues(lambda x : sum(x))\
                                           

In [10]:
zip_associated_with_multiple_cities_1.sortBy(lambda x : x[1], ascending = False).take(1)

[('94080', 40)]

#### v2

In [11]:
zip_associated_with_multiple_cities_2 = distinct_zip_city_pairs.map(lambda x : (x[0], 1))\
                                           .reduceByKey(lambda x, y : x + y)\
                                           

In [12]:
zip_associated_with_multiple_cities_2.sortBy(lambda x : x[1], ascending = False).take(1)

[('94080', 40)]

### Which one is more efficient? v1 or v2??

#### 

## 5. Create zip_city_count_names which includes (zip, (count, the list of names))

In [13]:
zip_city_count_names = distinct_zip_city_pairs.mapValues(lambda x: [x])\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda x: (x[0], (len(list(x[1])), sorted(list(x[1])))))\
    .sortBy(lambda x: x[1][0], ascending=False)

In [14]:
zip_city_count_names.take(1)

[('94080',
  (40,
   ['Daly City',
    'S San Fran',
    'S San Francisco',
    'S+san+fran',
    'S+san+francisco',
    'S. San Francisco',
    'S.+san+francisco',
    'San Francisco',
    'San+bruno',
    'San+carlos',
    'San+francisco',
    'So San Francisco',
    'So+.+san+francisco',
    'So+san+francisco',
    'So.+san+francisco',
    'So.san+francisco',
    'Soith+san+francisco',
    'Souht+san+francisco',
    'Sout+san+francisco',
    'South San Francisco',
    'South San Fransico',
    'South Sanfrancisco',
    'South++san+francisco',
    'South+san+fracisco',
    'South+san+fran',
    'South+san+franc',
    'South+san+francisc',
    'South+san+francisco',
    'South+san+francisico',
    'South+san+francsico',
    'South+san+franisco',
    'South+san+franscisco',
    'South+san+fransisco',
    'South+san.francisco',
    'South+sanfrancisco',
    'South+sanfranciso',
    'South+sanfransico',
    'South+sf',
    'Ssf',
    'Suth+san+francisco']))]

In [16]:
sc.stop()